# Consdb for Latiss 

- Creation date : 2026-01-21
- https://usdf-rsp-dev.slac.stanford.edu/consdb/
- Schemes for constdb : https://sdm-schemas.lsst.io/
- Documentation : https://consdb.lsst.io/index.html
- w_2026_03


In [ ]:
from lsst.summit.utils import ConsDbClient

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cmx
from matplotlib.colors import ListedColormap

import seaborn as sns

%matplotlib widget

from lsst.meas.algorithms.installGaussianPsf import FwhmPerSigma

from tqdm.notebook import tqdm

In [ ]:
from mysitcom.auxtel.qualitycuts import plot_single_param_vs_time

In [ ]:
# xx-small
# x-small
# small
# medium
# large
# x-large
# xx-large

plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

In [ ]:
from astropy.table import Table, join
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy import coordinates
import astropy.coordinates as coord
import astropy.units as u

In [ ]:
import lsst.geom as geom
import lsst
from lsst.geom import Angle

In [ ]:
# https://pipelines.lsst.io/modules/lsst.geom/getting-started.html
func_degToRad = lambda x: Angle(x, lsst.geom.degrees).asRadians()
func_zendtoAirmass = lambda x: 1.0 / np.cos(func_degToRad(x))
# func_wrap = lambda x : Angle(x,lsst.geom.radians).wrap(180.*lsst.geom.degrees)

In [ ]:
def ra_to_mollweide(ra_rad):
    """
    Transforme une ascension droite (RA en radians) pour une projection Mollweide :
    - RA doit être d'abord dans [0, 2π]
    - Puis ramené dans [-π, +π] pour correspondre au domaine Mollweide
    - Puis inversé pour que RA croisse vers la gauche sur la carte

    Paramètre:
    - ra_rad : array ou scalaire de RA en radians

    Retourne:
    - RA transformé pour projection Mollweide

    Discussion avec ChatGPT verifiée le 28/04/2025
    """

    # 1) Assurer que RA est entre 0 et 2π
    ra_rad = np.remainder(ra_rad + 2 * np.pi, 2 * np.pi)

    # 2) Remettre RA entre -π et +π
    ra_rad[ra_rad > np.pi] -= 2 * np.pi

    # 3) Inverser RA pour que RA croisse vers la gauche
    return -ra_rad

## Configuration

### Choose instrument

In [ ]:
instrument = "LATISS" # data since 20221001

In [ ]:
os.environ["no_proxy"] += ",.consdb"

In [ ]:
url = "http://consdb-pq.consdb:8080/consdb"

In [ ]:
consdb = ConsDbClient(url)

https://sdm-schemas.lsst.io/cdb_lsstcomcam.html#exposure

In [ ]:
exposures = consdb.query("SELECT * FROM cdb_latiss.exposure WHERE day_obs >= 20220101")
visits = consdb.query("SELECT * FROM cdb_latiss.visit1 WHERE day_obs >= 20220101")

In [ ]:
print(visits.columns)

In [ ]:
print(exposures.columns)

In [ ]:
df_exposures = exposures.to_pandas()
df_visits = visits.to_pandas()

In [ ]:
df_exposures.head()

In [ ]:
list(df_exposures.columns)

In [ ]:
col_sel_exp = ['exposure_id','target_name','obs_start','obs_start_mjd',
               'sky_rotation',
               'azimuth',
               'altitude',
               'zenith_distance',
               'airmass',
               'exp_time',
               'shut_time',
               'dark_time',
               'group_id',   
               'air_temp',
               'pressure',
               'humidity',
               'wind_speed',
               'wind_dir',
               'dimm_seeing',
               'shut_lower',
               'shut_upper',
               'focus_z',
               'dome_azimuth',
               'simulated',
               's_region',
               'vignette',
               'vignette_min',
               'can_see_sky',
               'science_program','observation_reason']

In [ ]:
cut_exposure = df_exposures["can_see_sky"] & (df_exposures["exp_time"] >= 30.) & \
    (df_exposures["physical_filter"].isin(["empty~holo4_003","BG40_65mm_1","OG550_65mm_1"])) & \
    (df_exposures["observation_reason"] != "acq")


In [ ]:
df_exposures[cut_exposure][col_sel_exp]['exp_time'].unique() 

In [ ]:
df_exposures_sel = df_exposures[cut_exposure][col_sel_exp].sort_values(by="exposure_id")
df_exposures_sel 

In [ ]:
df_exposures_sel.groupby("exp_time").size()

In [ ]:
df_exposures_sel["obs_start"] = pd.to_datetime(df_exposures_sel["obs_start"],format='ISO8601')


In [ ]:
plot_single_param_vs_time(
    df_exposures_sel,
    time_col="obs_start",
    param="exp_time",
    color="darkred",
    zoomrange=(0, 80),
)

In [ ]:
for param in col_sel_exp:
    if param not in ["target_name","Time","obs_start","group_id",'simulated','s_region','can_see_sky','science_program','observation_reason']:
        plot_single_param_vs_time(
            df_exposures_sel,
            time_col="obs_start",
            param=param,
            color="darkred",
            zoomrange=None
        )
    